##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Recommending movies: ranking

This tutorial is a slightly adapted version of the [basic ranking tutorial](https://www.tensorflow.org/recommenders/examples/basic_ranking) from TensorFlow Recommenders documentation.



## Imports


Let's first get our imports out of the way.

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

In [16]:
import os
import pprint
import tempfile
import datetime

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

## Preparing the dataset

We're continuing to use the MovieLens dataset. This time, we're also going to keep the ratings: these are the objectives we are trying to predict.

In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

We'll split the data by putting 80% of the ratings in the train set, and 20% in the test set.

In [4]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

Next we figure out unique user ids and movie titles present in the data so that we can create the embedding user and movie embedding tables.

In [5]:
movie_titles = ratings.batch(1_000_000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

## Implementing a model

### Architecture

Ranking models do not face the same efficiency constraints as retrieval models do, and so we have a little bit more freedom in our choice of architectures. We can implement our ranking model as follows:

In [6]:
class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])
    
  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

### Loss and metrics

We'll make use of the `Ranking` task object: a convenience wrapper that bundles together the loss function and metric computation. 

We'll use it together with the `MeanSquaredError` Keras loss in order to predict the ratings.

In [7]:
task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

### The full model

We can now put it all together into a model.

In [8]:
class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel()
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")
    
    rating_predictions = self(features)

    # The task computes the loss and the metrics.
    return self.task(labels=labels, predictions=rating_predictions)

## Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [10]:
model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.legacy.Adagrad(learning_rate=0.1))

Then shuffle, batch, and cache the training and evaluation data.

In [11]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

Then train the  model:

In [12]:
%load_ext tensorboard

In [22]:
!rm -rf ./logs/

In [25]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
%tensorboard --logdir logs/fit

In [26]:
model.fit(cached_train, epochs=100,callbacks=[tensorboard_callback])

Epoch 1/100
10/10 [==============================] - 0s 18ms/step - root_mean_squared_error: 0.9099 - loss: 0.8305 - regularization_loss: 0.0000e+00 - total_loss: 0.8305
Epoch 2/100
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.9098 - loss: 0.8302 - regularization_loss: 0.0000e+00 - total_loss: 0.8302
Epoch 3/100
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.9096 - loss: 0.8299 - regularization_loss: 0.0000e+00 - total_loss: 0.8299
Epoch 4/100
10/10 [==============================] - 0s 16ms/step - root_mean_squared_error: 0.9094 - loss: 0.8295 - regularization_loss: 0.0000e+00 - total_loss: 0.8295
Epoch 5/100
10/10 [==============================] - 0s 18ms/step - root_mean_squared_error: 0.9092 - loss: 0.8292 - regularization_loss: 0.0000e+00 - total_loss: 0.8292
Epoch 6/100
10/10 [==============================] - 0s 14ms/step - root_mean_squared_error: 0.9090 - loss: 0.8288 - regularization_loss: 0.0000e+00 -

As the model trains, the loss is falling and the RMSE metric is improving.

Finally, we can evaluate our model on the test set:

In [27]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 6ms/step - root_mean_squared_error: 0.9216 - loss: 0.8507 - regularization_loss: 0.0000e+00 - total_loss: 0.8507


{'root_mean_squared_error': 0.9216275215148926,
 'loss': 0.856576144695282,
 'regularization_loss': 0,
 'total_loss': 0.856576144695282}

The lower the RMSE metric, the more accurate our model is at predicting ratings.

## Exporting for serving

The model can be easily exported for serving:


In [29]:
tf.saved_model.save(model, "exported-ranking/123")

INFO:tensorflow:Assets written to: exported-ranking/123/assets


INFO:tensorflow:Assets written to: exported-ranking/123/assets



We will deploy the model with TensorFlow Serving soon.

In [30]:
# Zip the SavedModel folder for easier download
!zip -r exported-ranking.zip exported-ranking/ 

  adding: exported-ranking/ (stored 0%)
  adding: exported-ranking/123/ (stored 0%)
  adding: exported-ranking/123/fingerprint.pb (stored 0%)
  adding: exported-ranking/123/variables/ (stored 0%)
  adding: exported-ranking/123/variables/variables.data-00000-of-00001 (deflated 19%)
  adding: exported-ranking/123/variables/variables.index (deflated 62%)
  adding: exported-ranking/123/saved_model.pb (deflated 80%)
  adding: exported-ranking/123/assets/ (stored 0%)


In [32]:
test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
Dances with Wolves (1990): [[4.104224]]
Speed (1994): [[3.8703105]]
M*A*S*H (1970): [[3.868439]]
